<a href="https://colab.research.google.com/github/alin2025/My_Code_Example/blob/main/SQLAlchemy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [161]:
import pandas as pd
from sqlalchemy import create_engine, Table, MetaData, Column, Integer, String, Float,inspect
from sqlalchemy import select, desc
from sqlalchemy.sql import func
import sqlalchemy

> **Reference:** [Excellent SQLAlchemy tutorial with examples by Vinay Kudari](https://towardsdatascience.com/sqlalchemy-python-tutorial-79a577141a91)

# SQLAlchemy basic concepts

[**SQLAlchemy**](http://docs.sqlalchemy.org/en/latest/core/engines.html) provides a nice “Pythonic” way of interacting with databases. So rather than dealing with the differences between specific dialects of traditional SQL such as MySQL or PostgreSQL or Oracle, you can leverage the Pythonic framework of SQLAlchemy to streamline your workflow and more efficiently query your data.

In this example we will interact with an [**SQLite**](http://www.sqlitetutorial.net/) database, which is a C library that provides a lightweight disk-based database. We will work in front of an existing DB called [`chinook`](http://www.sqlitetutorial.net/sqlite-sample-database).

## Engine

[Engine](http://docs.sqlalchemy.org/en/latest/core/connections.html#sqlalchemy.engine.Engine) is the most fundamental object of SQLAlchemy, and it defines the database we work with.

In [145]:
engine = sqlalchemy.create_engine('sqlite:///chinook.db', echo=False)

In [162]:
Inspector=inspect(engine)
Inspector.get_table_names()

['albums',
 'artists',
 'customers',
 'employees',
 'genres',
 'invoice_items',
 'invoices',
 'media_types',
 'playlist_track',
 'playlists',
 'tracks']

## MetaData

Database tables in SQLAlchemy belong (are linked to) a [metadata](https://docs.sqlalchemy.org/en/13/core/connections.html#connectionless-execution-implicit-execution) object.

In [196]:
metadata = sqlalchemy.MetaData()

In [197]:
metadata.tables.keys()

dict_keys([])

## [`Table`](https://docs.sqlalchemy.org/en/13/core/metadata.html#sqlalchemy.schema.Table)

In [169]:
albums = Table('albums', metadata, autoload_with=engine)

In [179]:
type(albums)

sqlalchemy.sql.schema.Table

In [180]:
albums.c.keys()

['AlbumId', 'Title', 'ArtistId']

In [181]:
metadata.tables

FacadeDict({'albums': Table('albums', MetaData(), Column('AlbumId', INTEGER(), table=<albums>, primary_key=True, nullable=False), Column('Title', NVARCHAR(length=160), table=<albums>, nullable=False), Column('ArtistId', INTEGER(), ForeignKey('artists.ArtistId'), table=<albums>, nullable=False), schema=None), 'artists': Table('artists', MetaData(), Column('ArtistId', INTEGER(), table=<artists>, primary_key=True, nullable=False), Column('Name', NVARCHAR(length=120), table=<artists>), schema=None)})

> **Discussion:** What is the difference between Python's Table and SQL's Table? Mention the concept of DB API.

## Connection

The `connect()` method returns a [*Connection*][con] object, through which we can send commands to the database.

[con]: http://docs.sqlalchemy.org/en/latest/core/connections.html#sqlalchemy.engine.Connection "Connection docs"

In [182]:
conn = engine.connect()

# Executing DB operations

Using SQLAlchemy, the `Connection`'s [`execute(object_)`](https://docs.sqlalchemy.org/en/13/core/connections.html#sqlalchemy.engine.Connection.execute) method executes SQL commands in 2 optional ways, either by sending explicit SQL commands, or by wrapping them with Pythonic objects.

## SQL statements

A straight-forward approach would be to use our connection and "send" SQL commands.

In [183]:
query = text('''
SELECT * FROM albums
WHERE Title LIKE '%the best of%'
''')

In [184]:
result = conn.execute(query)

In [185]:
type(result)

sqlalchemy.engine.cursor.CursorResult

> **Note:** This is the one place Python is case insensitive...

The result is a [`ResultProxy`](https://docs.sqlalchemy.org/en/13/core/connections.html#sqlalchemy.engine.ResultProxy). which is an iterator.

In [186]:
result.fetchmany(5)

[(13, 'The Best Of Billy Cobham', 10),
 (20, 'The Best Of Buddy Guy - The Millenium Collection', 15),
 (47, 'The Best of Ed Motta', 37),
 (61, "Knocking at Your Back Door: The Best Of Deep Purple in the 80's", 58),
 (83, 'My Way: The Best Of Frank Sinatra [Disc 1]', 85)]

> **Note:** `ResultProxy` is an iterator. What will happen if we run `fetchmany()` again?

> **Discussion:** To execute queries this way you have to be proficient in SQL. Pros and cons...

## [`ClauseElement`](https://docs.sqlalchemy.org/en/latest/core/sqlelement.html#sqlalchemy.sql.expression.ClauseElement)

**The power of an API lies in its objects**, and SQLAlchemy provides "Pythonic" objects to represent SQL functionalities. More specifically, we will be interested in  [`FromClause`](https://docs.sqlalchemy.org/en/13/core/selectable.html#sqlalchemy.sql.expression.FromClause) elements.

> **Reference:** More information about the available expressions (virtually most of SQL functionality) can be found in this [SQL expression language tutorial by SQLAlchemy](https://docs.sqlalchemy.org/en/13/core/tutorial.html#sql-expression-language-tutorial).

### Example 1 - select

**Task - Show the names of the employees and their job title.**

We will demonstrate the API with the [`select()`](https://docs.sqlalchemy.org/en/13/core/selectable.html#sqlalchemy.sql.expression.select) method, which returns a [`Select`](https://docs.sqlalchemy.org/en/13/core/selectable.html?highlight=select#sqlalchemy.sql.expression.Select) object.

Let's demonstrate with the *employees* table.

In [198]:
employees = Table('employees', metadata, autoload_with=engine)

In [199]:
metadata.tables.keys()

dict_keys(['employees'])

In [188]:
type(employees)

sqlalchemy.sql.schema.Table

When we create a `select()` construct, SQLAlchemy looks around at the tables we’ve mentioned and then places them in the FROM clause of the statement. We can select the entire table or specific columns.

In [210]:
query = sqlalchemy.select(employees)


#query = select([employees.c.EmployeeId, employees.c.FirstName, employees.c.LastName, employees.c.Title])

# query = select(
#     employees.c.EmployeeId,
#     employees.c.FirstName,
#     employees.c.LastName,
#     employees.c.Title
# )

In [211]:
type(query)

sqlalchemy.sql.selectable.Select

> **Discussion:** Discuss the [`Select`](https://docs.sqlalchemy.org/en/13/core/selectable.html?highlight=select#sqlalchemy.sql.expression.Select) object and its available properties.

In [212]:
print(str(query))

SELECT employees."EmployeeId", employees."LastName", employees."FirstName", employees."Title", employees."ReportsTo", employees."BirthDate", employees."HireDate", employees."Address", employees."City", employees."State", employees."Country", employees."PostalCode", employees."Phone", employees."Fax", employees."Email" 
FROM employees


In [213]:
result = conn.execute(query)

In [214]:
type(result)

sqlalchemy.engine.cursor.CursorResult

In [ ]:
result.fetchall()

### Example 2 - select-where

**Task - Show the names of the albums which contain the phrase "The best of"**.

We can use the `whereclause` argument of `select()`

In [220]:
query = select(
    albums.c.Title
).where(
    albums.c.Title.like('%the best of%')
)

Alternatively, we can use the `where()` method of the `Select` object.

In [222]:
query = select(albums).where(albums.columns.Title.like('%the best of%'))

In [223]:
result = conn.execute(query)

In [224]:
result.fetchmany(5)

[(13, 'The Best Of Billy Cobham', 10),
 (20, 'The Best Of Buddy Guy - The Millenium Collection', 15),
 (47, 'The Best of Ed Motta', 37),
 (61, "Knocking at Your Back Door: The Best Of Deep Purple in the 80's", 58),
 (83, 'My Way: The Best Of Frank Sinatra [Disc 1]', 85)]

### Example 3 - join

**Task - Show the names of all the albums and their artists.**

In [225]:
artists = Table('artists', metadata, autoload_with=engine)
albums = Table('albums', metadata, autoload_with=engine)

We use the [`join()`](https://docs.sqlalchemy.org/en/13/core/metadata.html?highlight=join#sqlalchemy.schema.Table.join) method.

In [226]:
join_stmt = artists.join(albums, artists.c.ArtistId == albums.c.ArtistId)

In [227]:
print(type(join_stmt))

<class 'sqlalchemy.sql.selectable.Join'>


> **Note:** Both `select()` and `join()` are special cases of the more general [`FromClause`](https://docs.sqlalchemy.org/en/13/core/selectable.html#sqlalchemy.sql.expression.FromClause) class, which basically means they can be used within the FROM clause of a SELECT statement. This can be illustrated by looking at

>> `join_stmt.c.keys()`

>> `conn.execute(select([join_stmt])).fetchmany(5)`

Now we can select from this Join object.

In [229]:
query = select(
    albums.c.Title,
    artists.c.Name,
               from_obj=join_stmt)

ArgumentError: Additional keyword arguments are not accepted by this function/method.  The presence of **kw is for pep-484 typing purposes

In [230]:
import sqlalchemy
from sqlalchemy import select, join

# Create the engine
engine = sqlalchemy.create_engine('sqlite:///chinook.db', echo=False)

# Define the metadata
metadata = sqlalchemy.MetaData()

# Reflect the tables
metadata.reflect(bind=engine)

# Get the 'albums' and 'artists' tables from metadata
albums = metadata.tables['albums']
artists = metadata.tables['artists']

# Define the join condition
join_stmt = join(albums, artists, albums.c.ArtistId == artists.c.ArtistId)

# Establish a connection
conn = engine.connect()

# Define the query to select columns from both tables and join them
query = select(
    albums.c.Title,
    artists.c.Name
).select_from(
    join_stmt
)

# Execute the query on the connection
result = conn.execute(query)

# Fetch the results
for row in result:
    print(row)

# Close the connection
conn.close()


('For Those About To Rock We Salute You', 'AC/DC')
('Balls to the Wall', 'Accept')
('Restless and Wild', 'Accept')
('Let There Be Rock', 'AC/DC')
('Big Ones', 'Aerosmith')
('Jagged Little Pill', 'Alanis Morissette')
('Facelift', 'Alice In Chains')
('Warner 25 Anos', 'Antônio Carlos Jobim')
('Plays Metallica By Four Cellos', 'Apocalyptica')
('Audioslave', 'Audioslave')
('Out Of Exile', 'Audioslave')
('BackBeat Soundtrack', 'BackBeat')
('The Best Of Billy Cobham', 'Billy Cobham')
('Alcohol Fueled Brewtality Live! [Disc 1]', 'Black Label Society')
('Alcohol Fueled Brewtality Live! [Disc 2]', 'Black Label Society')
('Black Sabbath', 'Black Sabbath')
('Black Sabbath Vol. 4 (Remaster)', 'Black Sabbath')
('Body Count', 'Body Count')
('Chemical Wedding', 'Bruce Dickinson')
('The Best Of Buddy Guy - The Millenium Collection', 'Buddy Guy')
('Prenda Minha', 'Caetano Veloso')
('Sozinho Remix Ao Vivo', 'Caetano Veloso')
('Minha Historia', 'Chico Buarque')
('Afrociberdelia', 'Chico Science & Nação Z

Alternatively, when we use JOINs we know what FROM clause we want, so here we make use of the [`select_from()`](https://docs.sqlalchemy.org/en/13/core/selectable.html?highlight=select_from#sqlalchemy.sql.expression.Select.select_from) method.

In [ ]:
# query = select([albums.c.Title, artists.c.Name]).select_from(join_stmt)

In [ ]:
print(str(query))

SELECT albums."Title", artists."Name" 
FROM artists JOIN albums ON artists."ArtistId" = albums."ArtistId"


In [ ]:
result = conn.execute(query)

In [ ]:
result.fetchmany(5)

[('For Those About To Rock We Salute You', 'AC/DC'),
 ('Balls to the Wall', 'Accept'),
 ('Restless and Wild', 'Accept'),
 ('Let There Be Rock', 'AC/DC'),
 ('Big Ones', 'Aerosmith')]

### Example 4 - Group By and SQL functions

**Task - Show for each customer (name) the number of invoinces they had.**

> **Reference:** Very often we wish to apply a function on the data. Built-in SQL functions are available through the [`func` module](https://docs.sqlalchemy.org/en/13/core/tutorial.html#functions).

See here details about the [`group_by()`](https://docs.sqlalchemy.org/en/13/core/tutorial.html#ordering-grouping-limiting-offset-ing) method.

In [ ]:
invoices = Table('invoices', metadata, autoload_with=engine)
customers = Table('customers', metadata, autoload_with=engine)

Let's try first without the names of the customers.

In [ ]:
query = select([invoices.c.CustomerId, func.count(invoices.c.InvoiceId)])\
            .select_from(invoices)\
            .group_by(invoices.c.CustomerId)

In [ ]:
print(str(query))

SELECT invoices."CustomerId", count(invoices."InvoiceId") AS count_1 
FROM invoices GROUP BY invoices."CustomerId"


In [ ]:
conn.execute(query).fetchmany(50)

[(1, 7),
 (2, 7),
 (3, 7),
 (4, 7),
 (5, 7),
 (6, 7),
 (7, 7),
 (8, 7),
 (9, 7),
 (10, 7),
 (11, 7),
 (12, 7),
 (13, 7),
 (14, 7),
 (15, 7),
 (16, 7),
 (17, 7),
 (18, 7),
 (19, 7),
 (20, 7),
 (21, 7),
 (22, 7),
 (23, 7),
 (24, 7),
 (25, 7),
 (26, 7),
 (27, 7),
 (28, 7),
 (29, 7),
 (30, 7),
 (31, 7),
 (32, 7),
 (33, 7),
 (34, 7),
 (35, 7),
 (36, 7),
 (37, 7),
 (38, 7),
 (39, 7),
 (40, 7),
 (41, 7),
 (42, 7),
 (43, 7),
 (44, 7),
 (45, 7),
 (46, 7),
 (47, 7),
 (48, 7),
 (49, 7),
 (50, 7)]

And now with the names.

In [ ]:
query = select([customers.c.FirstName + " " + customers.c.LastName, func.count(invoices.c.InvoiceId)])\
            .select_from(invoices.join(customers, invoices.c.CustomerId == customers.c.CustomerId))\
            .group_by(invoices.c.CustomerId)

In [ ]:
conn.execute(query).fetchmany(5)

[('Luís Gonçalves', 7),
 ('Leonie Köhler', 7),
 ('François Tremblay', 7),
 ('Bjørn Hansen', 7),
 ('František Wichterlová', 7)]

## Relation to pandas

### Tables (SQLAlchemy) vs. DataFrames (pandas)

Very often we would like to save our query result as a data-frame. Luckily, the `ResultProxy.fetchall()` method returns a list of results that can be constructed as a data-frame using the standard `pd.DataFrame()` constructor.

In [ ]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


ERROR:sqlalchemy.pool.impl.NullPool:Exception during reset or similar
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sqlalchemy/pool/base.py", line 676, in _finalize_fairy
    fairy._reset(pool)
  File "/usr/local/lib/python3.10/dist-packages/sqlalchemy/pool/base.py", line 881, in _reset
    pool._dialect.do_rollback(self)
  File "/usr/local/lib/python3.10/dist-packages/sqlalchemy/engine/default.py", line 667, in do_rollback
    dbapi_connection.rollback()
sqlite3.ProgrammingError: SQLite objects created in a thread can only be used in that same thread. The object was created in thread id 140653444310848 and this is thread id 140652763395840.
ERROR:sqlalchemy.pool.impl.NullPool:Exception closing connection <sqlite3.Connection object at 0x7fec5568f040>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sqlalchemy/pool/base.py", line 676, in _finalize_fairy
    fairy._reset(pool)
  File "/usr/local/lib/python3.10/dist-pa

In [ ]:
query = select([albums])
result = conn.execute(query)

In [ ]:
df_albums = pd.DataFrame(result.fetchall(), columns=result.keys())
df_albums.head()

,AlbumId,Title,ArtistId
0,1,For Those About To Rock We Salute You,1
1,2,Balls to the Wall,2
2,3,Restless and Wild,2
3,4,Let There Be Rock,1
4,5,Big Ones,3


### Direct querying

Moreover, *pandas* offers [`pd.read_sql_query(sql, con)`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_sql_query.html) to directly send SQL commands through a given DB connection. SQLAlchemy engine is one of the options for `con`.

In [ ]:
sql = 'SELECT * FROM employees Order By BirthDate'
df_tracks = pd.read_sql_query(sql, con=engine)
df_tracks.head()

,EmployeeId,LastName,FirstName,Title,ReportsTo,BirthDate,HireDate,Address,City,State,Country,PostalCode,Phone,Fax,Email
0,4,Park,Margaret,Sales Support Agent,2.0,1947-09-19 00:00:00,2003-05-03 00:00:00,683 10 Street SW,Calgary,AB,Canada,T2P 5G3,+1 (403) 263-4423,+1 (403) 263-4289,margaret@chinookcorp.com
1,2,Edwards,Nancy,Sales Manager,1.0,1958-12-08 00:00:00,2002-05-01 00:00:00,825 8 Ave SW,Calgary,AB,Canada,T2P 2T3,+1 (403) 262-3443,+1 (403) 262-3322,nancy@chinookcorp.com
2,1,Adams,Andrew,General Manager,NaN,1962-02-18 00:00:00,2002-08-14 00:00:00,11120 Jasper Ave NW,Edmonton,AB,Canada,T5K 2N1,+1 (780) 428-9482,+1 (780) 428-3457,andrew@chinookcorp.com
3,5,Johnson,Steve,Sales Support Agent,2.0,1965-03-03 00:00:00,2003-10-17 00:00:00,7727B 41 Ave,Calgary,AB,Canada,T3B 1Y7,1 (780) 836-9987,1 (780) 836-9543,steve@chinookcorp.com
4,8,Callahan,Laura,IT Staff,6.0,1968-01-09 00:00:00,2004-03-04 00:00:00,923 7 ST NW,Lethbridge,AB,Canada,T1H 1Y8,+1 (403) 467-3351,+1 (403) 467-8772,laura@chinookcorp.com


In [ ]:
albums = pd.read_sql('select * from albums', con=engine)

In [ ]:
albums1 = pd.read_sql_table('albums',con=engine)

In [ ]:
albums1.drop('Title', axis=1).to_sql('albums_new', con=engine,if_exists='replace')

347

In [ ]:
albums_new = pd.read_sql('select * from albums_new', con=engine)
albums_new

,index,AlbumId,ArtistId
0,0,1,1
1,1,2,2
2,2,3,2
3,3,4,1
4,4,5,3
...,...,...,...
342,342,343,226
343,343,344,272
344,344,345,273
345,345,346,274


In [ ]:
engine.tab

AttributeError: ignored

In [ ]:
engine.table_names()

<ipython-input-101-d72bc75a6b62>:1: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  engine.table_names()


['albums',
 'albums_new',
 'artists',
 'customers',
 'employees',
 'genres',
 'invoice_items',
 'invoices',
 'media_types',
 'new_albums',
 'playlist_track',
 'playlists',
 'sqlite_sequence',
 'sqlite_stat1',
 'tracks']

In [ ]:
albums

,AlbumId,Title,ArtistId
0,1,For Those About To Rock We Salute You,1
1,2,Balls to the Wall,2
2,3,Restless and Wild,2
3,4,Let There Be Rock,1
4,5,Big Ones,3
...,...,...,...
342,343,Respighi:Pines of Rome,226
343,344,Schubert: The Late String Quartets & String Qu...,272
344,345,Monteverdi: L'Orfeo,273
345,346,Mozart: Chamber Music,274


In [ ]:
albums['agg'] = 1

In [ ]:
albums

,AlbumId,Title,ArtistId,agg
0,1,For Those About To Rock We Salute You,1,1
1,2,Balls to the Wall,2,1
2,3,Restless and Wild,2,1
3,4,Let There Be Rock,1,1
4,5,Big Ones,3,1
...,...,...,...,...
342,343,Respighi:Pines of Rome,226,1
343,344,Schubert: The Late String Quartets & String Qu...,272,1
344,345,Monteverdi: L'Orfeo,273,1
345,346,Mozart: Chamber Music,274,1


In [ ]:
albums.to_sql('new_albums', con=engine, if_exists='append')

347

In [ ]:
albums

,AlbumId,Title,ArtistId,agg
0,1,For Those About To Rock We Salute You,1,1
1,2,Balls to the Wall,2,1
2,3,Restless and Wild,2,1
3,4,Let There Be Rock,1,1
4,5,Big Ones,3,1
...,...,...,...,...
342,343,Respighi:Pines of Rome,226,1
343,344,Schubert: The Late String Quartets & String Qu...,272,1
344,345,Monteverdi: L'Orfeo,273,1
345,346,Mozart: Chamber Music,274,1


In [ ]:
new_df= pd.read_sql_table('new_albums', con=engine)

In [ ]:
new_df.head(1)

,index,AlbumId,Title,ArtistId,agg
0,0,1,For Those About To Rock We Salute You,1,1


In [ ]:
new_df['AlbumId'].count()

1041

In [ ]:
new_df['AlbumId']

0         1
1         2
2         3
3         4
4         5
       ... 
1036    343
1037    344
1038    345
1039    346
1040    347
Name: AlbumId, Length: 1041, dtype: int64

In [ ]:
new_df['AlbumId'].duplicated().sum()

694

> **Your turn:**
* Part 1 - create the tables `tracks`, `albums` and `artists` both as SQLAlchemy Tables and as pandas DataFrames.
* Part 2 - Answer the following questions in two ways - using SQLAlchemy and using pandas.
>> 1. What is the size of the table `tracks`?
>> 2. Which artist has the highest number of tracks?

> Don't hesitate to look for the answers online...

### Solution

#### Part 1

In [ ]:
tracks = Table('tracks', metadata, autoload_with=engine)
query = select([tracks])
results = conn.execute(query).fetchall()
df_tracks = pd.DataFrame(results, columns=tracks.c.keys())
df_tracks.head()

<ipython-input-56-e50fe691bb3f>:3: SAWarning: Dialect sqlite+pysqlite does *not* support Decimal objects natively, and SQLAlchemy must convert from floating point - rounding errors and other issues may occur. Please consider storing Decimal numbers as strings or integers on this platform for lossless storage.
  results = conn.execute(query).fetchall()


,TrackId,Name,AlbumId,MediaTypeId,GenreId,Composer,Milliseconds,Bytes,UnitPrice
0,1,For Those About To Rock (We Salute You),1,1,1,"Angus Young, Malcolm Young, Brian Johnson",343719,11170334,0.99
1,2,Balls to the Wall,2,2,1,None,342562,5510424,0.99
2,3,Fast As a Shark,3,2,1,"F. Baltes, S. Kaufman, U. Dirkscneider & W. Ho...",230619,3990994,0.99
3,4,Restless and Wild,3,2,1,"F. Baltes, R.A. Smith-Diesel, S. Kaufman, U. D...",252051,4331779,0.99
4,5,Princess of the Dawn,3,2,1,Deaffy & R.A. Smith-Diesel,375418,6290521,0.99


In [ ]:
albums = Table('albums', metadata, autoload_with=engine)
query = select([albums])
results = conn.execute(query).fetchall()
df_albums = pd.DataFrame(results, columns=albums.c.keys())
df_albums.head()

,AlbumId,Title,ArtistId
0,1,For Those About To Rock We Salute You,1
1,2,Balls to the Wall,2
2,3,Restless and Wild,2
3,4,Let There Be Rock,1
4,5,Big Ones,3


In [ ]:
artists = Table('artists', metadata, autoload_with=engine)
query = select([artists])
results = conn.execute(query).fetchall()
df_artists = pd.DataFrame(results, columns=artists.c.keys())
df_artists.head()

,ArtistId,Name
0,1,AC/DC
1,2,Accept
2,3,Aerosmith
3,4,Alanis Morissette
4,5,Alice In Chains


#### Part 2

##### Question 1

Based on [`count()` documentation](https://docs.sqlalchemy.org/en/13/core/functions.html#sqlalchemy.sql.functions.count)

In [ ]:
query = select([func.count()]).select_from(tracks)
conn.execute(query).fetchall()

[(3503,)]

Or...

In [ ]:
len(df_tracks)

3503

##### Question 2

In [ ]:
join_stmt = tracks.join(albums, tracks.c.AlbumId == albums.c.AlbumId)\
    .join(artists, albums.c.ArtistId == artists.c.ArtistId)

Based on [`order_by()`](https://docs.sqlalchemy.org/en/13/core/selectable.html?highlight=order_by#sqlalchemy.sql.expression.Select.order_by), [`desc()`](https://docs.sqlalchemy.org/en/13/core/sqlelement.html?highlight=desc#sqlalchemy.sql.expression.desc) and [`label()`](https://docs.sqlalchemy.org/en/13/core/sqlelement.html#sqlalchemy.sql.expression.label)  documentation.

In [ ]:
query = select([artists.c.Name, func.count(tracks.c.TrackId).label('tracks_count')])\
    .select_from(join_stmt)\
    .group_by(artists.c.ArtistId)\
    .order_by(desc('tracks_count'))

In [ ]:
conn.execute(query).fetchmany(5)

[('Iron Maiden', 213),
 ('U2', 135),
 ('Led Zeppelin', 114),
 ('Metallica', 112),
 ('Deep Purple', 92)]

Or (with the help of the [`DataFrame.join()` documentation](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.join.html))...

In [ ]:
df_all = df_tracks\
    .join(df_albums.set_index('AlbumId'), on='AlbumId')\
    .join(df_artists.set_index('ArtistId'), on='ArtistId',
          lsuffix='_l', rsuffix='_r')
df_all.Name_r.value_counts()[:5]

Iron Maiden     213
U2              135
Led Zeppelin    114
Metallica       112
Lost             92
Name: Name_r, dtype: int64

# Create your own DB

## Table creation

For this tutorial we will use an in-memory-only SQLite database. This is an easy way to test things without needing to have an actual database defined anywhere.

In [ ]:
engine = create_engine('sqlite:///:memory:', echo=False)

We have to define a metadata object.

In [ ]:
metadata = MetaData()

Next we define the schemas of the tables.

In [ ]:
users = Table('users', metadata,
    Column('id', Integer),
    Column('name', String),
    Column('fullname', String),
)

In [ ]:
addresses = Table('addresses', metadata,
    Column('id', Integer),
    Column('user_id', Integer),
    Column('email_address', String)
)

> **Note:** The metadata object makes sure there are no ambiguities in the database. Try to create another table with the same name and read the exception.

Finally, we use the metadata object to create all the tables.

In [ ]:
metadata.create_all(engine)

## Insert data

All operations are sent to the database through the connection object.

In [ ]:
conn = engine.connect()

The [`insert()`](https://docs.sqlalchemy.org/en/13/core/dml.html#sqlalchemy.sql.expression.insert) method is a wrapper for SQL's INSERT command.

In [ ]:
ins = users.insert().values(id=1234, name='jack', fullname='Jack Jones')

In [ ]:
str(ins)

'INSERT INTO users (id, name, fullname) VALUES (:id, :name, :fullname)'

In [ ]:
result = conn.execute(ins)

> **Note:** This `ResultProxy` object does not contain any result.

### Testing

In [ ]:
conn.execute(select([users])).fetchall()

[(1234, 'jack', 'Jack Jones')]

# Example

In this example we do the following:
1. We insert the data of the MovieLens files into two database tables.
2. We use SQLAlchemy to find the best movie (having at least 30 viewers).

## Inspect the data

In [ ]:
import sys
if 'google.colab' in sys.modules:
    from google.colab import files
    uploaded = files.upload()

In [ ]:
df_movies = pd.read_csv('movies.csv')
df_movies.head()

FileNotFoundError: File b'movies.csv' does not exist

In [ ]:
df_ratings = pd.read_csv('ratings.csv')
df_ratings.head()

## Creating the tables

In [ ]:
engine = create_engine('sqlite:///:memory:', echo=False)
metadata = MetaData()
conn = engine.connect()

In [ ]:
movies = Table('movies', metadata,
    Column('movieID', Integer),
    Column('title', String),
    Column('genres', String),
)

In [ ]:
ratings = Table('ratings', metadata,
    Column('userID', Integer),
    Column('movieID', Integer),
    Column('rating', Float),
    Column('timestamp', Integer)
)

In [ ]:
metadata.create_all(engine)

## Inserting the data

In [ ]:
for ind, row in df_movies.iterrows():
    ins = movies.insert().values(movieID=row.movieID, title=row.title, genres=row.genres)
    conn.execute(ins)

In [ ]:
conn.execute(select([movies])).fetchmany(5)

In [ ]:
for ind, row in df_ratings.iterrows():
    ins = ratings.insert().values(userID=row.userID, movieID=row.movieID, rating=row.rating, timestamp=row.timestamp)
    conn.execute(ins)

In [ ]:
conn.execute(select([ratings])).fetchmany(5)

## Executing the query

In [ ]:
join_stmt = movies.join(ratings, ratings.c.movieID == movies.c.movieID)

**To Be Continued**

In [ ]:
engine = create_engine('mssql+pyodbc://{}:{}@'.format(server_user, server_password) \
                            + server_name + '/' + \
                            db_name + '?trusted_connection=no&driver=ODBC+Driver+17+for+SQL+server')